<a href="https://colab.research.google.com/github/engromersharif/manning_chatbot_1_3/blob/main/QnA_Bot_Wikipwdia_Telegram.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyTelegramBotAPI
!pip install wikipedia
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from transformers import pipeline

qna = pipeline('question-answering', model='deepset/gelectra-base-germanquad', token='deepset/gelectra-base-germanquad')

def answer_question(qna, quest, context):
    answ = qna({'question': quest, 'context' : context} )
    return answ['answer'] + f" Confidence: {answ['score']}"

In [3]:
answer_question(qna,"Wie alt ist Kurt?","Der Name des Vaters lautet Kurt und er ist 28 Jahre alt")

'28 Jahre Confidence: 0.3824869692325592'

In [4]:
import wikipedia
def create_context_from_wiki(lang, page):
  wikipedia.set_lang(lang)
  topic = wikipedia.page(page)
  return(topic.content)



In [5]:
create_context_from_wiki('de','flugzeug')

'Ein Flugzeug ist ein Luftfahrzeug, das schwerer als Luft ist und den zu seinem Fliegen nötigen dynamischen Auftrieb mit nicht-rotierenden Auftriebsflächen erzeugt. In der enger gefassten Definition der Internationalen Zivilluftfahrtorganisation (ICAO) ist es auch immer ein motorisiertes Luftfahrzeug. Der Betrieb von Flugzeugen, die am Luftverkehr teilnehmen, wird durch Luftverkehrsgesetze geregelt.\nUmgangssprachlich werden Flugzeuge mitunter auch Flieger genannt, der Ausdruck Flieger hat als Hauptbedeutung jedoch den Piloten.\n\n\n== Definition ==\nDie Internationale Zivilluftfahrtorganisation (International Civil Aviation Organization, ICAO) definiert den Begriff Flugzeug wie folgt:\n\nIm rechtlichen Sprachgebrauch ist ein Flugzeug ein motorgetriebenes Luftfahrzeug, schwerer als (die von ihm verdrängte) Luft, das seinen Auftrieb durch Tragflächen erhält, die bei gleichbleibenden Flugbedingungen unverändert bleiben, allgemeinsprachlich Motorflugzeug genannt. Wenn in einem Gesetzestex

In [6]:
def response_answer(input):
  return('Nenne mir ein Thema über das Du reden möchtest?')

In [7]:
response_answer('Hi')

'Nenne mir ein Thema über das Du reden möchtest?'

In [8]:
import telebot

bot = telebot.TeleBot("", parse_mode=None) 

state = 0
context = ''
question = ''

@bot.message_handler(commands=['start', 'help'])
def send_welcome(message):
	bot.reply_to(message, "Howdy, how are you doing?")

@bot.message_handler(func=lambda message: True)
def echo_all(message):
    global state
    global context
    if state == 0:
      bot.reply_to(message, response_answer(message.text))
      state = 1
      return

    if state == 1:
      context = create_context_from_wiki('de', message.text)
      bot.reply_to(message, "Stell mir eine Frage zu dem Thema?")
      state = 2
      return

    if state == 2:
      answ = answer_question(qna,message.text, context)
      bot.reply_to(message, answ)
      state = 0
      return

bot.polling()